# Predict Helix Capping Residues #

The goal is to identify residues just before an alpha helix begins or the residues just after the helix ends. This will improve secondary structure predictors becuase they often extend too far or do not start at the right place. 

The CapsDB has annoted sequences of structures of helix capping residues that can be used to train a deep nueral net. We will use a Bidirectional LSTM using phi/psi features to see if it will those will be good predictors.

## 1. Download data ##

## 2. Generate Features ##
### MMTF Pyspark Imports ###

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.webfilters import Pisces
from mmtfPyspark.filters import ContainsLProteinChain
from mmtfPyspark.mappers import StructureToPolymerChains
from mmtfPyspark.ml import ProteinSequenceEncoder
import numpy as np
import pandas as pd
import math

### Custom imports ###

In [2]:
import secondaryStructureExtractorFull
#import mmtfToASA
import os
os.getcwd()

'/home/ec2-user/SageMaker/ProteinFragmenter'

### Configure Spark Context ###

In [3]:
spark = SparkSession.builder.master("local[8]").appName("DeepCap").getOrCreate()

### Filter out chains not in CapsDB ###

In [4]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import concat, col, lit, array_contains

sqlContext = SQLContext(spark)
capsdb = sqlContext.read.parquet('caps_descriptors.parquet')
capsdb_pdbs = capsdb.select(concat(upper(col("pdbId")), lit("."), col("chain")).alias("id")).drop_duplicates()


In [ ]:
capsdb_pdbs.count()

In [5]:
#from pyspark.sql.types import BooleanType

# def hasHelixCapInfo(enc_obj, ids):
#     return(enc_obj[0] in ids)

# def udf_hasHelixCapInfo(ids):
#     return udf(lambda t: hasHelixCapInfo(t, ids))
#     # I'm not sure how to call these successfully - Sean

capsdb_ids = set()
[capsdb_ids.add(list[0]) for list in capsdb_pdbs.select("id").collect()]

class HasHelixCapInfo(object):
    '''This filter returns true if the structure is in CAPSDB'''
    def __call__(self, t):
        return(t[0] in capsdb_ids)
    # This works but I don't like that it refers to a global variable: not sure how to change that right now - Sean


### Read MMTF File and get a set of L-protein chains ###

In [ ]:
if not os.path.isdir("full"):
    !wget https://mmtf.rcsb.org/v1.0/hadoopfiles/full.tar && tar -xvf full.tar
        
if not os.path.isdir("reduced"):
    !wget https://mmtf.rcsb.org/v1.0/hadoopfiles/reduced.tar && tar -xvf reduced.tar

In [ ]:
pdb = mmtfReader.read_sequence_file('full') \
    .flatMap(StructureToPolymerChains()) \
    .filter(ContainsLProteinChain()) \
    .filter(HasHelixCapInfo())
#pdb.count()
#pdb.take(2)
#?pdb

### Get Torsion angle and secondary structure info ###

In [ ]:
#from mmtfPyspark.datasets import secondaryStructureExtractor

data = secondaryStructureExtractorFull.get_dataset(pdb).toPandas()
#data = secondaryStructureExtractor.get_dataset(pdb).toPandas()

In [6]:
# Read output of above get_dataset operation from parquet file
parquetPath = '/home/ec2-user/SageMaker/ProteinFragmenter/datacaps.parquet'
dataframe = sqlContext.read.parquet(parquetPath)
data = dataframe.toPandas()
data = data.drop('__index_level_0__', axis=1)

In [7]:
data.head(10)

,pdbId,chain,resi,resn,phi,psi
0,2ygn,A,1,THR,NaN,163.677383
1,2ygn,A,2,GLY,-66.660973,160.703186
2,2ygn,A,3,SER,-123.853607,-7.871733
3,2ygn,A,4,LEU,-74.896896,137.483932
4,2ygn,A,5,TYR,-134.419830,140.864288
5,2ygn,A,6,LEU,-139.275024,127.621544
6,2ygn,A,7,TRP,-152.167755,166.833832
7,2ygn,A,8,ILE,-108.079048,119.799377
8,2ygn,A,9,ASP,-61.786110,150.193756
9,2ygn,A,10,ALA,-47.469296,-38.584801


In [8]:
df1 = capsdb.toPandas()
df = pd.merge(data, df1, left_on=('pdbId','chain'), right_on=('pdbid','chain'), how='inner')
df = df[['pdbId', 'chain', 'resi', 'resn', 'phi', 'psi', 'startcap', 'endcap']]


### Create labels

In [12]:
df['is_cap'] = df.apply(lambda x: 1 if (x['resi'] >= x['startcap'] and x['resi'] <= x['endcap']) else 0, axis=1)
df_caps = df.groupby(["pdbId", "chain", "resi"])['is_cap'].max().reset_index()

In [13]:
data_caps = pd.merge(data, df_caps, left_on=('pdbId','chain', 'resi'), right_on=('pdbId','chain', 'resi'), how='inner')

In [14]:
from Bio.PDB.Polypeptide import aa3
one_hot_encoded = pd.DataFrame(data_caps.resn.apply(lambda x: secondaryStructureExtractorFull.get_residue(x)).tolist(), columns=aa3)
one_hot_encoded.head()
data_caps = data_caps.join(one_hot_encoded)
data_caps.head()

,pdbId,chain,resi,resn,phi,psi,is_cap,ALA,CYS,ASP,...,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
0,2ygn,A,1,THR,NaN,163.677383,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2ygn,A,2,GLY,-66.660973,160.703186,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2ygn,A,3,SER,-123.853607,-7.871733,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,2ygn,A,4,LEU,-74.896896,137.483932,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2ygn,A,5,TYR,-134.419830,140.864288,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
data_caps.head()

In [15]:
#test = training_data[0:128,:,:]

def is_cap(pdbId, chain, resi, is_cap):
    if is_cap == 1:
        return([1,0])
    elif is_cap == 0:
        return([0,1])
    else:
        raise ValueError("is_cap must be 0 or 1")

def angle_to_cos(angle):
    if(angle == 0 or np.isnan(angle)):
        return 0
    else:
        #print("working")
        return np.cos(np.pi * angle/180)

def angle_to_sin(angle):
    if(angle == 0 or np.isnan(angle)):
        return 0
    else:
        #print("working")
        return np.sin(np.pi * angle/180)


In [16]:
groups = data_caps.groupby(["pdbId", "chain"])
                           # num pdbs,    max len of seqs, num features
training_data = np.zeros((groups.ngroups, 5000, 24), dtype=float)
                           # num pdbs,    max len of seqs, length of one-hot encoded target
truth = np.zeros((groups.ngroups, 5000, 2), dtype=int)
truth_lagged = np.zeros((groups.ngroups, 5000, 2), dtype=int)


In [32]:
for i, ((pdbid, chain), group) in enumerate(groups):
    for j, featuretuple in enumerate(group.itertuples()):
        if j>=5000: break
        #truth[i,j,:] = is_cap(featuretuple.pdbId, featuretuple.chain, featuretuple.resi, featuretuple.is_cap)
        training_data[i,j,:] = (angle_to_cos(featuretuple.phi), angle_to_sin(featuretuple.phi), 
                              angle_to_cos(featuretuple.psi), angle_to_sin(featuretuple.psi), featuretuple.ALA,
                              featuretuple.CYS,featuretuple.ASP,featuretuple.GLU,featuretuple.PHE,
                              featuretuple.GLY,featuretuple.HIS,featuretuple.ILE,featuretuple.LYS,
                              featuretuple.LEU,featuretuple.MET,featuretuple.ASN,featuretuple.PRO,
                              featuretuple.GLN,featuretuple.ARG,featuretuple.SER,featuretuple.THR,
                              featuretuple.VAL,featuretuple.TRP,featuretuple.TYR)
        #if (j > 0):
            #truth_lagged[i,j-1,:] = truth[i,j,:]
        

In [33]:
training_data[10,2,:]
#truth[2]
#truth[8,:,0].sum()
#truth.shape
#training_data.shape
#os.getcwd()

array([-0.55415247, -0.83241518, -0.9238779 ,  0.38268737,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ])

In [4]:
# get subset of training data, if needed for faster debugging

#training_data = HDF5Matrix('features.h5', 'training_data')
import h5py
hf = h5py.File('features.h5', 'r')
training_data = hf.get('training_data').value
hf.close()

hf = h5py.File('truth.h5', 'r')
truth = hf.get('truth').value
hf.close()

hf = h5py.File('truthlag.h5', 'r')
truth_lagged = hf.get('truthlag').value
hf.close()

/home/ec2-user/anaconda3/envs/myBinder-environment/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


### Write features, labels to H5 file ###

In [34]:
import h5py
os.remove('features.h5')
os.remove('truth.h5')
os.remove('truthlag.h5')

h5f = h5py.File('features.h5', 'w')
h5f.create_dataset('training_data', data=training_data)
h5f.close()

h5tr = h5py.File('truth.h5', 'w')
h5tr.create_dataset('truth', data=truth)
h5tr.close()

h5trl = h5py.File('truthlag.h5', 'w')
h5trl.create_dataset('truthlag', data=truth_lagged)
h5trl.close()

#os.remove("features.h5")

### Terminate Spark ###

In [31]:
spark.stop()

## 4. Build Bidirectional LSTM ##

In [2]:
def create_and_train_model(num_features, num_outputs=2, latent_dim=100):
    """Create a Seq2Seq Bidirectional LSTM
    From: https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
    
    Parameters
    ----------
    num_features : int
        The number of features in your trianing data
    num_outputs : int
        Number of outputs to rpedict, i.e. number of classes or 2 for binary
        
    Returns
    -------
    A new Keras Seq2Seq Bidirectional LSTM
    """
    
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(None, num_features))
    encoder = CuDNNLSTM(latent_dim, return_state=True) #, activation='relu')
    #keras.layers.add([encoder, bn])
    
    #encoder = Bidirectional(LSTM(latent_dim, return_state=True))
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    bn = BatchNormalization()(encoder_outputs)
    #encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
    #state_h = Concatenate()([forward_h, backward_h])
    #state_c = Concatenate()([forward_c, backward_c])
        
    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    #decoder_inputs = Input(shape=(None, num_outputs))
        
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    #decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    #decoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True))
    #decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    
    decoder_inputs = Input(shape=(None, num_outputs))
    decoder_lstm = CuDNNLSTM(latent_dim, return_sequences=True, return_state=True) #, activation='relu')
    #decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    #bn_decoder = BatchNormalization()(decoder_outputs)
    
    decoder_dense = Dense(num_outputs, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # summarize layers
    print(model.summary())
    
    # Train model
    encoder_input_data = HDF5Matrix('features.h5', 'training_data')
    decoder_input_data = HDF5Matrix('truth.h5', 'truth')
    decoder_target_data = HDF5Matrix('truthlag.h5', 'truthlag')
    
    #Automicatlly determine batch sizes, train/test splits
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data, shuffle="batch", epochs=10) #, callbacks=get_callbacks(model))
    for (i, obj) in enumerate(model.get_weights()):
        obj.tofile("weights_layer{0}".format(str(i)))
    
    # Create encoder/decoder model
    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    #decoder_model.add(BatchNormalization())
    
    return encoder_model, decoder_model

In [4]:
def get_callbacks(model_obj, logging_file="training.log", verbosity=1, early_stopping_patience=None):
    callbacks = list()
    callbacks.append(ModelCheckpoint(model_obj, save_best_only=True))
    callbacks.append(CSVLogger(logging_file, append=True))
#    callbacks.append(LossHistory('losshistory.log', append=True))
    callbacks.append(TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='batch'))
#     if learning_rate_epochs:
#         callbacks.append(LearningRateScheduler(partial(step_decay, initial_lrate=initial_learning_rate,
#                                                        drop=learning_rate_drop, epochs_drop=learning_rate_epochs)))
#     else:
#         callbacks.append(ReduceLROnPlateau(factor=learning_rate_drop, patience=learning_rate_patience,
#                                            verbose=verbosity))
#     if early_stopping_patience:
#         callbacks.append(EarlyStopping(verbose=verbosity, patience=early_stopping_patience))
    return callbacks

In [ ]:
#encoder_input_data = HDF5Matrix('features.h5', 'training_data')
training_data.tofile("test.txt")

In [ ]:
# get subset of training data, if needed for faster debugging

#training_data = HDF5Matrix('features.h5', 'training_data')
import h5py
hf = h5py.File('full_dataset_features/features.h5', 'r')
training_data = hf.get('training_data').value
training_data = training_data[0:128,:,:]
hf.close()

hf = h5py.File('full_dataset_features/truth.h5', 'r')
truth = hf.get('truth').value
truth = truth[0:128,:,:]
hf.close()

hf = h5py.File('full_dataset_features/truthlag.h5', 'r')
truth_lagged = hf.get('truthlag').value
truth_lagged = truth_lagged[0:128,:,:]
hf.close()

In [ ]:
#import numpy
#numpy.set_printoptions(threshold=numpy.nan)


In [3]:
import tensorflow as tf
import keras
from keras.utils.io_utils import HDF5Matrix
from keras.layers import Input, Dense, Bidirectional, LSTM, Concatenate, CuDNNLSTM
from keras.models import Model
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard
from keras.layers.normalization import BatchNormalization
from keras.utils import plot_model


# class LossHistory(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.losses = []

#     def on_batch_end(self, batch, logs={}):
#         self.losses.append(logs.get('loss'))

#encoder_inputs, encoder_states, decoder_inputs, decoder_lstm, decoder_dense, model = train()
#model = train()
encoder_model, decoder_model = create_and_train_model(num_features=4, num_outputs=2, latent_dim=100)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 4)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 2)      0                                            
__________________________________________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)        [(None, 100), (None, 42400       input_1[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)        [(None, None, 100),  41600       input_2[0][0]                    
                                                                 cu_dnnlstm_1[0][1]               
          

KeyboardInterrupt: 

In [135]:
encoder_model.save_weights("encoder_weights")
decoder_model.save_weights("decoder_weights")

In [ ]:
# load json and create model
json_file = open('model_StS_BiLSTM.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_StS_BiLSTM.h5")
print("Loaded model from disk")

In [155]:
np.fromfile("weights_layer5")

array([-7.93595155e-15, -4.27936601e-13, -6.64940235e-16, -3.91690746e-13,
       -5.23724873e-14,  2.45457441e-14, -2.61107594e-13, -5.63939015e-14,
        3.23818823e-13, -1.28629050e-14, -7.93595155e-15, -4.27936601e-13,
        6.28694050e-03,  4.46223932e-03,  5.16536179e-03,  2.45457441e-14,
       -2.61107594e-13, -5.63939015e-14,  3.23818823e-13, -1.28629050e-14])

In [102]:
truth[0,:,:]

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

In [158]:
def decode_sequence(input_seq, encoder_model, decoder_model):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    #print(states_value)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, 2))
    # Populate the first character of target sequence with the start residue.
    target_seq[0, 0, :] = [0, 1] # decoder tokens one-hot encoding hardcoded here

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_chain = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        print(output_tokens)
        sampled_token_index = (np.argmax(output_tokens[0, -1, :]) + 1) % 2
        decoded_chain.append(sampled_token_index) # flip indices

        # Exit condition: either hit max length
        # or find stop character.
        if len(decoded_chain) >= 5000:
            stop_condition = True

        # Update the target sequence (of length 1).
        sampled_residue = [0, 0]
        sampled_residue[sampled_token_index] = 1
        target_seq = np.zeros((1, 1, 2))
        target_seq[0, 0, :] = sampled_residue

        # Update states
        states_value = [h, c]

    return decoded_chain

In [143]:
# To Sean: ran til here

In [84]:
#training_data = HDF5Matrix('features.h5', 'training_data')
import h5py
hf = h5py.File('features.h5', 'r')
df_training = hf.get('training_data').value
hf.close()

# import h5py
# hf = h5py.File('truth.h5', 'r')
# df_truth = hf.get('truth').value
# hf.close()

/home/ec2-user/anaconda3/envs/myBinder-environment/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [162]:
temp = df_training[4:5,:,:]

In [166]:
result = decode_sequence(temp, encoder_model, decoder_model)

[[[0.47100836 0.5289916 ]]]
[[[0.50156456 0.49843544]]]
[[[0.4749491 0.5250509]]]
[[[0.50743854 0.49256146]]]
[[[0.474107 0.525893]]]
[[[0.5112709  0.48872918]]]
[[[0.4745381 0.525462 ]]]
[[[0.51503146 0.48496854]]]
[[[0.47577474 0.5242253 ]]]
[[[0.51819885 0.48180112]]]
[[[0.4769654  0.52303463]]]
[[[0.52047944 0.4795206 ]]]
[[[0.47778144 0.5222185 ]]]
[[[0.52194536 0.47805458]]]
[[[0.478225 0.521775]]]
[[[0.52280885 0.47719112]]]
[[[0.47840625 0.5215937 ]]]
[[[0.52327937 0.47672066]]]
[[[0.4784362 0.5215638]]]
[[[0.5235149  0.47648507]]]
[[[0.4783938  0.52160615]]]
[[[0.5236197  0.47638023]]]
[[[0.47832632 0.5216737 ]]]
[[[0.5236566 0.4763434]]]
[[[0.47825792 0.5217421 ]]]
[[[0.52366096 0.47633898]]]
[[[0.47819886 0.5218012 ]]]
[[[0.5236518  0.47634825]]]
[[[0.47815192 0.5218481 ]]]
[[[0.5236383 0.4763617]]]
[[[0.47811648 0.52188355]]]
[[[0.5236249 0.4763751]]]
[[[0.4780906 0.5219094]]]
[[[0.5236133 0.4763867]]]
[[[0.47807217 0.52192783]]]
[[[0.5236039  0.47639605]]]
[[[0.47805926 0.

[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[

[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[

[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[

[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]


...

[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]


[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]


[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[

[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]


[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]
[[[0.4780339 0.5219661]]]
[[[0.52357924 0.47642076]]]


In [167]:
i = 0
for j in result:
    i += j
i

2500

In [168]:
result

[0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
